# 1.1 By Hadamard Test
## Multiple Experiments

Consider a scenario, you have multiple circuits that you want to run at once.

Call `.measure()` one by one will be inefficient,
no to mention that you also need to call `.anlyze()` for their post-processing.

Here we provide a more efficient way solve this problem,
where the true power of Qurrium as experiment manage toolkit.


### a. Import the instances


In [1]:
from qurry import EntropyMeasure

experiment_hadamard = EntropyMeasure(method="hadamard")

### b. Preparing quantum circuit

Prepare and add circuits to the `.wave` for later usage.


In [2]:
from qiskit import QuantumCircuit
from qurry.recipe import TrivialParamagnet, GHZ


def make_neel_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(0, n, 2):
        qc.x(i)
    return qc


for i in range(2, 8, 2):
    experiment_hadamard.add(TrivialParamagnet(i), f"trivial_paramagnet_{i}")
    experiment_hadamard.add(GHZ(i), f"ghz_{i}")
    experiment_hadamard.add(make_neel_circuit(i), f"neel_{i}")

experiment_hadamard.waves

WaveContainer({
  'trivial_paramagnet_2': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x76f0c525bbc0>,
  'ghz_2': <qurry.recipe.simple.cat.GHZ object at 0x76f0c4f08410>,
  'neel_2': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x76f1ad056c90>,
  'trivial_paramagnet_4': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x76f0c507a630>,
  'ghz_4': <qurry.recipe.simple.cat.GHZ object at 0x76f0c5022810>,
  'neel_4': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x76f0c51174a0>,
  'trivial_paramagnet_6': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x76f0c4f21d60>,
  'ghz_6': <qurry.recipe.simple.cat.GHZ object at 0x76f0c4f21df0>,
  'neel_6': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x76f0c4f22000>})

### c. Execute multiple experiments at once

Let's demonstrate the true power of Qurrium.


In [3]:
from qurry.qurrent.hadamard_test.arguments import EntropyMeasureHadamardMeasureArgs

Preparing a configuration list for multiple experiments with following parameters:

```python
class EntropyMeasureHadamardMeasureArgs(total=False):
    """Output arguments for :meth:`output`."""
    shots: int
    """Number of shots."""
    tags: Optional[tuple[str, ...]]
    """The tags to be used for the experiment."""
    wave: Optional[Union[QuantumCircuit, Hashable]]
    """The key or the circuit to execute."""
    degree: Optional[Union[int, tuple[int, int]]]
    """The degree range."""
```


In [4]:
config_list: list[EntropyMeasureHadamardMeasureArgs] = [
    {
        "shots": 1024,
        "wave": f"{wave_names}_{i}",
        "degree": i // 2,
        "tags": (wave_names, f"size_{i}", f"system_range_{i//2}"),
    }
    for _ in range(10)
    for i in range(2, 8, 2)
    for wave_names in ["trivial_paramagnet", "ghz", "neel"]
]
print(len(config_list))

90


The `.multiOutput` will return an id of this `multimanager` instance,
which can be used to get the results and post-process them.

Each `multimanager` will export the experiments in a folder you can specify
by setting `save_location` parameter with default location for current directory
where Python executed.
It will create a folder with the name of the `multimanager` instance,
and inside it will create a folder for storing each experiment data.

It will do firstly in the building process, but you can skip it by setting `skip_build_write=True` to save time.
After all experiments are executed, it will export secondly,
which can also be skipped by setting `skip_output_write=True` for no files output.


In [5]:
multi_exps1 = experiment_hadamard.multiOutput(
    config_list,
    summoner_name="qurrent.hadamard_test",  # you can name it whatever you want
    multiprocess_build=True,
    # Using multiprocessing to build the experiments,
    # it will be faster but take all the CPU
    skip_build_write=True,
    # Skip the writing of the experiment as files during the build,
    save_location=".",
    # Save the experiment as files in the current directory
    multiprocess_write=True,
    # Writing the experiment as files using multiprocessing,
)
multi_exps1

| MultiManager building...
| Write "qurrent.hadamard_test.001", at location "qurrent.hadamard_test.001"


| 0/90   0%|          | - MultiManager building... - 00:00 < ?

| MultiOutput running...


| 0/90   0%|          | -  - 00:00 < ?

| Auto analysis is called, running analysis...


...

| Export multi.config.json for 16d8d617-1745-4e39-92ba-4e00d696fd45


| 0/89 - Exporting experiments... - 00:00 < ?

| 0/90 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurrent.hadamard_test.001/qurryinfo.json...
| Exporting qurrent.hadamard_test.001/qurryinfo.json done.


'16d8d617-1745-4e39-92ba-4e00d696fd45'

You can check the result of `multiOutput` that we just executed by accessing the `.multimanagers`

In [6]:
experiment_hadamard.multimanagers

MultiManagerContainer(num=1, {
  "16d8d617-1745-4e39-92ba-4e00d696fd45":
    <MultiManager(name="qurrent.hadamard_test.001", jobstype="local", ..., exps_num=90)>,
})

In [7]:
experiment_hadamard.multimanagers[multi_exps1]

<MultiManager(id="16d8d617-1745-4e39-92ba-4e00d696fd45",
  name="qurrent.hadamard_test.001",
  tags=(),
  jobstype="local",
  pending_strategy="tags",
  last_events={
    'output.001': '2025-06-17 19:48:17',
    'auto_report': '2025-06-17 19:48:17',},
  exps_num=90)>

### d. Run post-processing at once


In [8]:
experiment_hadamard.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=True,
    multiprocess_write=False,
)

| 0/90 - Analysis:  - 00:00 < ?

| "report.002" has been completed.


'16d8d617-1745-4e39-92ba-4e00d696fd45'

In [ ]:
print("| Available results:")
for k, v in (
    experiment_hadamard.multimanagers[multi_exps1]
    .quantity_container["report.001"]
    .items()
):
    print("| -", k, "with length", len(v))

| Available results:
| - ('trivial_paramagnet', 'size_4', 'system_range_2') with length 10
| - ('ghz', 'size_4', 'system_range_2') with length 10
| - ('neel', 'size_4', 'system_range_2') with length 10
| - ('trivial_paramagnet', 'size_2', 'system_range_1') with length 10
| - ('ghz', 'size_2', 'system_range_1') with length 10
| - ('neel', 'size_2', 'system_range_1') with length 10
| - ('trivial_paramagnet', 'size_6', 'system_range_3') with length 10
| - ('ghz', 'size_6', 'system_range_3') with length 10
| - ('neel', 'size_6', 'system_range_3') with length 10


- Example of the content of `quantity_container`

In [ ]:
experiment_hadamard.multimanagers[multi_exps1].quantity_container["report.001"][
    ("trivial_paramagnet", "size_4", "system_range_2")
][:2]

[{'purity': 1.0,
  'entropy': np.float64(-0.0),
  'input': {},
  'header': {'serial': 1, 'datetime': '2025-06-17 19:48:27', 'log': {}}},
 {'purity': 1.0,
  'entropy': np.float64(-0.0),
  'input': {},
  'header': {'serial': 1, 'datetime': '2025-06-17 19:48:27', 'log': {}}}]

### e. Run post-processing at once with specific analysis arguments

At first, we need to get the each experiment's id in the `multimanager` instance.


In [10]:
expkeys_of_multi_exps1 = list(
    experiment_hadamard.multimanagers[multi_exps1].exps.keys()
)
print(len(expkeys_of_multi_exps1))

90


1. If you want to run the post-processing for some specific experiments,
   for example, the first 3 experiments we get for the `multimanager` instance.


In [11]:
experiment_hadamard.multiAnalysis(
    summoner_id=multi_exps1,
    analysis_name="first_3",
    skip_write=True,
    multiprocess_write=False,
    specific_analysis_args={k: idx < 3 for idx, k in enumerate(expkeys_of_multi_exps1)},
)

| 90/90 - Analysis: Skipped 408bd454-e9c1-4f41-8d5c-fb5e65d5af2f in faa4a131-af5e-40ae-8cbf-354915d90cad. - 00:00 < 00:00

| "first_3.002" has been completed.


'faa4a131-af5e-40ae-8cbf-354915d90cad'

In [12]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_hadamard.multimanagers[multi_exps1]
        .quantity_container["first_3.002"]
        .values()
    ),
)

| Available results:
| length: 3


2. Or manually specify all the analysis arguments for each experiment.


In [13]:
experiment_hadamard.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=False,
    analysis_name="all_manual",
    multiprocess_write=True,
    specific_analysis_args={k: {} for idx, k in enumerate(expkeys_of_multi_exps1)},
)

| 90/90 - Analysis: Calculating entangled entropy:  - 00:00 < 00:00


| "all_manual.003" has been completed.
| Export multimanager...


| 9/9 - Exporting done - 00:00 < 00:00                       
| 3/3 - exported quantity complete - 00:00 < 00:00    

| Export multi.config.json for faa4a131-af5e-40ae-8cbf-354915d90cad



| 89/89 - Exporting experiments... - 00:03 < 00:00
| 90/90 - Loading file infomation... - 00:00 < 00:00

| Exporting file taglist...
| Exporting qurrent.hadamard_test.001/qurryinfo.json...
| Exporting qurrent.hadamard_test.001/qurryinfo.json done.


'faa4a131-af5e-40ae-8cbf-354915d90cad'

In [14]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_hadamard.multimanagers[multi_exps1]
        .quantity_container["all_manual.003"]
        .values()
    ),
)

| Available results:
| length: 90


### f. Read exported multimanager data


In [15]:
multi_exps1_reades = experiment_hadamard.multiRead(
    save_location=".",
    summoner_name="qurrent.hadamard_test.001",
)

| Retrieve qurrent.hadamard_test.001...
| at: qurrent.hadamard_test.001


| 90/90 100%|██████████| - Loading 90 experiments ... - 00:04 < 00:00


---

### Post-Process Availablities and Version Info

In [17]:
from qurry.process import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurry version: 0.12.2.dev2
---------------------------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust   JAX   
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes    No    
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes    No    
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes    No    
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes    No    
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes    No    
 - magnet_square
   - magnsq_core ............................ Yes    No     No     No  